In [9]:
import os
from glob import glob
from datetime import datetime

from scipy import interpolate
from pytmatrix import orientation, radar, tmatrix_aux, refractive
from pytmatrix.psd import PSDIntegrator, GammaPSD
from pytmatrix.tmatrix import TMatrix, Scatterer
from pytmatrix.tmatrix_psd import TMatrixPSD, GammaPSD

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit

import common

In [10]:
file_list = sorted(glob('/g/data/kl02/jss548/PST/disdrometer/Brisbane/*.txt'))

In [11]:
def brisbane_disdro_to_radar_moments(infile):
    
    DBZ_list = []
    ZDR_list = []
    KDP_list = []
    ATTEN_list = []
    TIME_list = []
    RAIN_list = []

    #parse text files
    data_line_len = 50
    dsd_dictlist = []
    print('processing', infile)
    with open(infile) as f:
        #init
        bin_name_list = []
        bin_value_list = []
        bin_mean_drop = []
        #parse date from filename
        fn = os.path.basename(txt_ffn)
        year = '20' + fn[1:3]
        doy = fn[3:6]
        #parse each line
        for line in f:
            #skip empty lines
            if len(line) == 1:
                continue
            elif len(line) < data_line_len:
                #parse rainfall
                rainfall = float(line)
                dsd_dict = {'dt_start':line_dt_start, 'dt_end':line_dt_end, 'rainfall':rainfall,
                            'bin_mean_drop':bin_mean_drop, 'bin_name_list':bin_name_list, 'bin_value_list':bin_value_list}
                dsd_dictlist.append(dsd_dict)
                #init
                bin_name_list = []
                bin_value_list = []
                bin_mean_drop = []
            else:
                #parse line
                line_list = line.split()
                #parse date
                line_dt_start = datetime.strptime(year + '-' + doy + ' ' + ':'.join(line_list[0:3]),'%Y-%j %H:%M:%S')
                line_dt_end = datetime.strptime(year + '-' + doy + ' ' + ':'.join(line_list[3:6]),'%Y-%j %H:%M:%S')
                #parse bin
                bin_mean_drop.append((float(line_list[6])+float(line_list[7]))/2)
                bin_name_list.append('-'.join(line_list[6:8]) + 'mm')
                bin_value_list.append(float(line_list[8]))
                
    #scattering calcs
    for item in dsd_dictlist:
        nd = np.array(item['bin_value_list'])
        mean_diam_drop_class = np.array(item['bin_mean_drop'])  

        if np.sum(nd) == 0:
            continue

        #calc radar moments
        dbz, zdr, kdp, atten_spec = common.scatter_off_2dvd_packed(mean_diam_drop_class, nd, scatterer)
        DBZ_list.append(dbz)
        ZDR_list.append(zdr)
        KDP_list.append(kdp)
        ATTEN_list.append(atten_spec)
        TIME_list.append(item['dt_start'])
        RAIN_list.append(item['rainfall'])
        
    return DBZ_list, ZDR_list, KDP_list, ATTEN_list, TIME_list, RAIN_list

In [12]:
# for cant in range(1, 31):
# print(cant)
band = 'C'
cant = 10 #wdith of canting angle distribution
temperature = 0 #used for refractive calculations
# Main T-matrix parameters initialisation
# Radar band in mm.
if band == 'S':
    radar_band = tmatrix_aux.wl_S
elif band == 'C':
    radar_band = tmatrix_aux.wl_C
# Scatterer class from pytmatrix
# tmatrix_aux.wl_C is the wavelength in mm
# refractive.m_w_10C is a dictionnary containing the refractive index (at 10C) for different wavelength.
if temperature == 0:
    scatterer = Scatterer(wavelength=radar_band, m=refractive.m_w_0C[radar_band])
elif temperature == 10:
        scatterer = Scatterer(wavelength=radar_band, m=refractive.m_w_10C[radar_band])
elif temperature == 20:
        scatterer = Scatterer(wavelength=radar_band, m=refractive.m_w_20C[radar_band])
        
scatterer.or_pdf = orientation.gaussian_pdf(cant)
scatterer.orient = orientation.orient_averaged_fixed    

# PSDIntegrator classfrom pytmatrix
scatterer.psd_integrator = PSDIntegrator()    

# Defining the axis ratio of drops.
scatterer.psd_integrator.D_max = 8
scatterer.psd_integrator.geometries = (tmatrix_aux.geom_horiz_back, tmatrix_aux.geom_horiz_forw)

##### !!!!HERE!!!! ######
scatterer.psd_integrator.axis_ratio_func = common.bzv_model
##### !!!!HERE!!!! ######

scatterer.psd_integrator.init_scatter_table(scatterer)

In [13]:
#init lists
DBZ_total = []
ZDR_total = []
KDP_total = []
ATTEN_total = []
TIME_total = []
RAIN_total = []

for infile in file_list:
    #load file and run pytmatrix
    DBZ_out, ZDR_out, KDP_out, ATTEN_out, TIME_out, RAIN_out = brisbane_disdro_to_radar_moments(infile)
    DBZ_total.extend(DBZ_out)
    ZDR_total.extend(ZDR_out)
    KDP_total.extend(KDP_out)
    ATTEN_total.extend(ATTEN_out)
    TIME_total.extend(TIME_out)
    RAIN_total.extend(RAIN_out)
    
dbz_array = np.array(DBZ_total)
zdr_array = np.array(ZDR_total)
kdp_array = np.array(KDP_total)
att_array = np.array(ATTEN_total)
time_array = np.array(TIME_total)
rain_array = np.array(RAIN_total)

zdr_db_array = 10*np.log10(zdr_array)

processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V08325_1.txt
processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V08330_1.txt
processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V08339_1.txt
processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V08340_1.txt
processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V08344_1.txt
processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V08346_1.txt
processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V09022_1.txt
processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V09026_1.txt
processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V09029_1.txt
processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V09032_1.txt
processing /g/data/kl02/jss548/PST/disdrometer/Brisbane/V09033_1.txt


In [14]:
np.savez(f'pytmatrix_out/brisbane_radarsim_{band}_{cant}deg_{temperature}C.npz', rain_array=rain_array, dbz_array=dbz_array, zdr_db_array=zdr_db_array, kdp_array=kdp_array, att_array=att_array, time_array = time_array)

In [8]:
a = [1,2,3]
b = [4,5,6]

a.extend(b)

print(a)

[1, 2, 3, 4, 5, 6]
